Here I will construct a CNN within tensorflow to identify the type of flower, won't actually be running this code here though becuase it requires a GPU, so that'll happen on a remote server but let's get the basics down here!


https://www.kaggle.com/alxmamaev/flowers-recognition

In [6]:
%matplotlib inline


import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import os

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [13]:
daisies = []
for filename in os.listdir('/Users/noahkasmanoff/MachLearn/Phys476/hw4/flowers/daisy'):
    #print(filename)
    img = cv2.imread('/Users/noahkasmanoff/MachLearn/Phys476/hw4/flowers/daisy/' + filename)
    daisies.append(img)

In [37]:
maxone = 0

for i in range(len(daisies)):
    daisy = daisies[i]
    size = np.shape(daisy)[1] #*np.shape(daisy)[1]
    if size>=maxone:
        print(i)
#         print(size)
        maxone = size 


0
1
3
5
7
11
12
13
14
15
17
19
22
23
24
25
26
29
30
31
33
35
41
45
49
54
56
59
60
63
64
69
74
75
76
77
78
83
85
86
91
94
96
100
103
105
106
107
108
109
112
113
117
118
119
120
125
143
146
147
148
153
157
161
162
164
167
168
170
172
174
175
182
186
188
189
193
195
203
204
209
210
219
222
229
233
236
241
243
245
248
249
256
262
263
269
271
273
275
276
283
286
287
288
291
315
407
420
421
422
423
427
428
431
438
441
446
448
449
518


In [38]:
np.shape(daisies[518])

(231, 640, 3)

In [3]:
filename =  '100080576_f52e8ee070_n.jpg'

In [8]:
print(type(filename))

<class 'str'>


In [2]:
cd flowers/daisy/

/Users/noahkasmanoff/MachLearn/Phys476/hw4/flowers/daisy


In [ ]:
img

In [ ]:
255-img

In [ ]:
print(filename)
img=cv2.imread(filename)
plt.imshow(img)

So basically the cells above convert this into an array, (1066, 1599, 3)  3 representing RGB vals, so that's on track with this flower recog software

Next up is to design the CNN... 

Pulling from Aymerican Dreams...




In [ ]:
np.shape(img)

In [ ]:
#should convert vals to float 32, will get on that later.
# Training Parameters
learning_rate = 0.001
num_steps = 500
batch_size = 128
display_step = 10

# Network Parameters
num_input = np.shape(img)[0]*np.shape(img)[1]  # Flower data input (img shape: x*y)
num_classes = 5 #total flower classes, daisy, etc. 5 total 
dropout = 0.5 # Dropout, probability to keep units

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [ ]:

# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    
    ###### Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, , 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [ ]:
 Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()